In [ ]:
# 데이터 전처리
# 1. null 처리
# 2. 불필요한 속성 처리
# 3. 인코딩 수행

# 모델 학습 및 검증/예측/평가
# 1. 결정트리/랜덤포레스트/로지스틱 회귀 학습 비교
# 2. k 폴드 교차 검증
# 3. cross_val_score() / GridSearchCV() 수행

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# 데이터 프레임 가져오기
titanic_df = pd.read_csv('./titanic_train.csv')
titanic_df.head(3)

In [ ]:
# 데이터 정보 확인 및 null 값 확인
titanic_df.info()

# mean과 median 비교
titanic_df.describe().T 

# object 컬럼타입 추출
titanic_df.dtypes[titanic_df.dtypes == 'object'].index.tolist()

# 컬럼 값의 첫 글자만 가져오기
titanic_df['Cabin'] = titanic_df['Cabin'].str[:1]

# 나이는 연속적인 변수이기 때문에 카테고리를 나누는 함수 생성
def get_category(age):
    cat = ''
    if age <= -1: cat = 'Unknown'
    elif age <= 5: cat = 'Baby'
    elif age <= 12: cat = 'Child'
    elif age <= 18: cat = 'Teenager'
    elif age <= 25: cat = 'Student'
    elif age <= 35: cat = 'Young Adult'
    elif age <= 60: cat = 'Adult'
    else: cat = 'Elderly'
               
    return cat

plt.figure(figsize=(10,8))

# 정렬을 하기 위해 ordering을 할 리스트 정하기
group_names = ['Unknown', 'Baby', 'Child', 'Teenager', 
               'Student', 'Young Adult', 'Adult', 'Elderly']

# lambda 적용하여 새로운 컬럼 생성
titanic_df['Age_cat'] = titanic_df['Age'].apply(lambda x: get_category(x))

# 도식화
sns.barplot(
    x='Age_cat',
    y='Survived',
    hue='Sex',
    data=titanic_df,
    order=group_names
)

# 카테고리에 관한 그래프 그리고, 해당 컬럼 삭제하기
titanic_df.drop('Age_cat', axis=1, inplace=True)

In [ ]:
# 레이블 인코딩 해주는 함수

from sklearn.preprocessing import LabelEncoder

def encode_features(dataDF):
    features = ['Cabin', 'Sex', 'Embarked']
    le = LabelEncoder()
    for feature in features:
        le.fit(dataDF[feature]) # 따로따로 fit
        dataDF[feature] = le.transform(dataDF[feature]) # 그 다음 transform
        
    return dataDF


titanic_df = encode_features(titanic_df)
titanic_df.head()

In [ ]:
## 데이터 preprocessin 과정을 한번에 다 해주는 함수

# Null 처리 함수
def fillna(df):
    df['Age'].fillna(df['Age'].mean(),inplace=True)
    df['Cabin'].fillna('N',inplace=True)
    df['Embarked'].fillna('N',inplace=True)
    return df

# 머신러닝 알고리즘에 불필요한 피처 제거
def drop_features(df):
    df.drop(['PassengerId','Name','Ticket'], axis=1, inplace=True)
    return df

# 레이블 인코딩 수행. 
def format_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin','Sex','Embarked']
    for feature in features:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

# 앞에서 설정한 Data Preprocessing 함수 호출
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

In [2]:
###############################
# 머신 러닝 수행 과정
###############################

# 1. 원본 데이터를 재로딩하고, feature 데이터 셋과 label 데이터셋 추출
titanic_df = pd.read_csv('./titanic_train.csv')
y_titanic_df = titanic_df['Survived']
X_titanic_df = titanic_df.drop('Survived', axis=1)

X_titanic_df = transform_features(X_titanic_df)

# 2. train 셋과 test 셋 나누기
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df,
                                                     test_size=.2, random_state=11)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# 3. DecisionTreeClassfier, RandomForestClassifier, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score

dt_clf = DecisionTreeClassifier(random_state=11)
rf_clf = RandomForestClassifier(random_state=11)
lr_clf = LogisticRegression(solver='liblinear')

# DecisionTreeClassifier 학습/예측/평가
dt_clf.fit(X_train, y_train)
dt_pred = dt_clf.predict(X_test)
print("DecisionTreeClassifier 정확도 : {0: .4f}".format(accuracy_score(y_test, dt_pred)))

# RandomForestClassifier 
rf_clf.fit(X_train, y_train)
rf_pred = rf_clf.predict(X_test)
print("RandomForestClassifier 정확도 : {0: .4f}".format(accuracy_score(y_test, rf_pred)))

# LogisticRegression
lr_clf.fit(X_train, y_train)
lr_pred = lr_clf.predict(X_test)
print("Logistic Regression 정확도 : {0: .4f}".format(accuracy_score(y_test ,lr_pred)))

# 4. kfold로 하기
# kfold는 인덱스를 반환해줌

from sklearn.model_selection import KFold

def exec_kfold(clf, folds=5):
    # 폴드 세트를 5개인 KFold 객체를 생성, 폴드 수만큼 예측결과 저장을 위한 리스트 객체 생성.
    kfold = KFold(n_splits=folds)
    scores = []
    
    # Kfold 교차검증
    for iter_count, (train_index, test_index) in enumerate(kfold.split(X_titanic_df)):
        # X_titanic_df 데이터에서 교차 검증별로 학습과 검정 데이터를 가리키는 index 생성
        X_train, X_test = X_titanic_df.values[train_index], X_titanic_df.values[test_index]
        y_train, y_test = y_titanic_df.values[train_index], y_titanic_df.values[test_index]
        
        # Classifier 학습/예측/정확도 계산
        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        scores.append(accuracy)
        print("교차 검증 {0} 정확도 : {1: .4f}".format(iter_count, accuracy))
        
    # 5개 fold에서의 평균 정확도 계산
    mean_score = np.mean(scores)
    print("평균 정확도 : {0: .4f}".format(mean_score))
    
exec_kfold(dt_clf, folds=5)

NameError: name 'pd' is not defined

In [ ]:
# 5. 위의 과정을 한번에 해주는 cross_val_score
# stratified fold 방식으로 해줌

from sklearn.model_selection import cross_val_score

scores = cross_val_score(dt_clf, X_titanic_df, y_titanic_df, cv=5)
for iter_count, accuracy in enumerate(scores):
    print("교차 검증 {0} 정확도 : {1: .4f}".format(iter_count, accuracy))
    
print("평균 정확도 : {0: .4f}".format(np.mean(scores)))

In [ ]:
# 6. GridSearchCV로 해주기

from sklearn.model_selection import GridSearchCV

parameters = {
    'max_depth' : [2,3,5,10],
    'min_samples_split' : [2,3,5],
    'min_samples_leaf' : [1,5,8]
}

grid_dclf = GridSearchCV(
    dt_clf, 
    param_grid=parameters,
    scoring='accuracy',
    cv=5
)
grid_dclf.fit(X_train, y_train)

print('GridSearchCV 최적 하이퍼 파라미터 : {0}'.format(grid_dclf.best_params_))
print('GridSearchCV 최고 정확도: {0: .4f}'.format(grid_dclf.best_score_))
# 최적의 estimator를 가지고 predict
best_dclf = grid_dclf.best_estimator_

dpredictions = best_dclf.predict(X_test)
accuracy = accuracy_score(y_test, dpredictions)
print('테스트 세트에서의 DecisionTreeClassifier 정확도 : {0: .4f}'.format(accuracy))